In [1]:
#importing required library
import keras
import os
import glob
import pandas as pd
import numpy as np
import cv2

Using TensorFlow backend.
C:\Users\saira\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\saira\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\saira\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\saira\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [33]:
#preprocessing the image data
file_dir1 = 'input1/*'
file_dir2 = 'input2/*'
files = glob.glob(file_dir1)
print(len(files))
x1 = []
for f1 in files:
      img_dir = os.path.join(f1)
  #print('img_dir',img_dir)
      images = glob.glob(img_dir)
      for img in images:
        img_temp = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
        img_temp = cv2.resize(img_temp,(500,200))
        img_temo = img_temp/255
        x1.append(img_temp)
import numpy as np
x1 = np.array(x1)
print(x1.shape)

250
(250, 200, 500)


In [39]:
#%%
files = glob.glob(file_dir2)
x2 = []
for f1 in files:
  img_dir = os.path.join(f1)
  print('img_dir',img_dir)
  images = glob.glob(img_dir)
  
  for img in images:
    img_temp = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
    img_temp = cv2.resize(img_temp,(500,200))
    img_temo = img_temp/255
    x2.append(img_temp)
import numpy as np
x2 = np.array(x2)
print(x2.shape)

img_dir input2\000.jpg
img_dir input2\001.jpg
img_dir input2\002.jpg
img_dir input2\003.jpg
img_dir input2\004.jpg
img_dir input2\005.jpg
img_dir input2\006.jpg
img_dir input2\007.jpg
img_dir input2\008.jpg
img_dir input2\009.jpg
img_dir input2\010.jpg
img_dir input2\011.jpg
img_dir input2\012.jpg
img_dir input2\013.jpg
img_dir input2\014.jpg
img_dir input2\015.jpg
img_dir input2\016.jpg
img_dir input2\017.jpg
img_dir input2\018.jpg
img_dir input2\019.jpg
img_dir input2\020.jpg
img_dir input2\021.jpg
img_dir input2\022.jpg
img_dir input2\023.jpg
img_dir input2\024.jpg
img_dir input2\025.jpg
img_dir input2\026.jpg
img_dir input2\027.jpg
img_dir input2\028.jpg
img_dir input2\029.jpg
img_dir input2\030.PNG
img_dir input2\031.PNG
img_dir input2\032.PNG
img_dir input2\033.PNG
img_dir input2\034.PNG
img_dir input2\035.PNG
img_dir input2\036.PNG
img_dir input2\037.PNG
img_dir input2\038.PNG
img_dir input2\039.PNG
img_dir input2\040.PNG
img_dir input2\041.PNG
img_dir input2\042.PNG
img_dir inp

In [28]:
ylabel = 'ylabel.csv'
y = pd.read_csv(ylabel)


In [29]:
from sklearn.utils import shuffle
x1,x2,y = shuffle(x1,x2,y,random_state=2)

# splitting dataset into train test
x1_train = x1[0:180]
x1_train = x1_train.reshape(180,200,500,1)
x1_val = x1[180:210]
x1_val = x1_val.reshape(30,200,500,1)

x2_train = x2[0:180]
x2_train = x2_train.reshape(180,200,500,1)
x2_val = x2[180:210]
x2_val = x2_val.reshape(30,200,500,1)

x1_test = x1[210 :]
x1_test = x1_test.reshape(40,200,500,1)
x2_test = x2[210 :]
x2_test = x2_test.reshape(40,200,500,1)

y_train = y[0:180]
y_val = y[180:210]
y_test = y[210 :]

In [30]:
def get_siamese_model(input_shape):
    #input_shape = (200,500,1)
    left_input=Input(input_shape)
    right_input=Input(input_shape)
    
    model = Sequential()
    model.add(Conv2D(32,(10,10), activation='relu', input_shape=input_shape,
                    kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(MaxPooling2D())
    model.add(Conv2D(64,(7,7), activation='relu',
                    kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(MaxPooling2D())   
    model.add(Conv2D(64,(4,4), activation='relu', 
                    kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(MaxPooling2D())
    model.add(Conv2D(128,(4,4), activation='relu',
                    kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(1024,activation='sigmoid',kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    
    encoded_l=model(left_input)
    encoded_r=model(right_input)
    
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0]-tensors[1]))
    L1_distance=L1_layer([encoded_l,encoded_r])
    
    prediction=Dense(1,activation='sigmoid')(L1_distance)
    
    siamese_net1 = Model(inputs=[left_input,right_input],outputs=prediction)
    return siamese_net1


In [31]:
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Lambda, Flatten, Dense

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K
K.set_image_data_format('channels_last')

In [ ]:
model = get_siamese_model((200,500,1))
optimizer = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print("starting training process")
print("...............................")
#t_start = time.time()
model.fit([x1_train,x2_train],y_train,verbose=1,batch_size=5,validation_data=[[x1_val,x2_val],y_val],epochs =20,shuffle = True)
scores = model.evaluate([x1_test,x2_test],y_test,verbose=1)
print(scores)
model.save('siamese_net.h5')
#t_stop = (time.time()-t_start())
#print(t_stop)

starting training process
...............................
Train on 180 samples, validate on 30 samples
Epoch 1/20
180/180 [==============================] - 645s 4s/step - loss: 12.0537 - accuracy: 0.5889 - val_loss: 5.5547 - val_accuracy: 0.5333
Epoch 2/20
180/180 [==============================] - 591s 3s/step - loss: 3.8424 - accuracy: 0.5500 - val_loss: 3.1274 - val_accuracy: 0.6000
Epoch 3/20
180/180 [==============================] - 505s 3s/step - loss: 2.7140 - accuracy: 0.5500 - val_loss: 2.3995 - val_accuracy: 0.4667
Epoch 4/20
180/180 [==============================] - 485s 3s/step - loss: 2.4141 - accuracy: 0.6056 - val_loss: 2.2774 - val_accuracy: 0.7000
Epoch 5/20
180/180 [==============================] - 517s 3s/step - loss: 2.1329 - accuracy: 0.6722 - val_loss: 1.9940 - val_accuracy: 0.5333
Epoch 6/20
180/180 [==============================] - 506s 3s/step - loss: 1.9050 - accuracy: 0.6833 - val_loss: 1.9234 - val_accuracy: 0.6000
Epoch 7/20
180/180 [==================